<a href="https://colab.research.google.com/github/PieterDujardin/NER/blob/master/ColabNotebooks/i2b2_2010/LSTM_CRF_w_embeddings_flair_on_i2b22010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install flair

     |████████████████████████████████| 143kB 5.8MB/s 
     |████████████████████████████████| 983kB 34.7MB/s 
     |████████████████████████████████| 798kB 41.4MB/s 
     |████████████████████████████████| 573kB 45.2MB/s 
     |████████████████████████████████| 256kB 42.7MB/s 
     |████████████████████████████████| 1.0MB 38.6MB/s 
     |████████████████████████████████| 890kB 33.7MB/s 
     |████████████████████████████████| 3.7MB 39.6MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=a0a2c287051810cbbeaa0c450c019da29c9861e780ce9a0476a6a0b6aba202fa
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=6ff31fce45d635bd990aed1efab4a34680592c4aaa9c7df90818c48f909162ac
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for sqlitedict:

In [0]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings
from typing import List
import flair

from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

import torch


In [0]:
# the default behavior is that the model gets put on GPU if available and runs on CPU if there is no GPU.
torch.cuda.is_available()

True

In [0]:
columns = {0: 'text', 1: 'ner'}
data_folder = 'drive/My Drive/Colab Notebooks/#2_BERT_on_i2b2_2010/data/conll2003format/'

In [0]:
corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')

2020-04-29 11:23:21,222 Reading data from drive/My Drive/Colab Notebooks/#2_BERT_on_i2b2_2010/data/conll2003format
2020-04-29 11:23:21,223 Train: drive/My Drive/Colab Notebooks/#2_BERT_on_i2b2_2010/data/conll2003format/train.txt
2020-04-29 11:23:21,224 Dev: drive/My Drive/Colab Notebooks/#2_BERT_on_i2b2_2010/data/conll2003format/dev.txt
2020-04-29 11:23:21,225 Test: drive/My Drive/Colab Notebooks/#2_BERT_on_i2b2_2010/data/conll2003format/test.txt


In [0]:
# len(corpus.test)+len(corpus.train)+len(corpus.dev)
corpus.train.sentences[0]

Sentence: "After being transferred from the interventional radiology table to the stretcher , one of his lateral percutaneous drains had been pulled out inadvertently ." - 24 Tokens

In [0]:
tag_type = 'ner'
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

In [0]:
print(tag_dictionary)

Dictionary with 10 tags: <unk>, O, B-treatment, I-treatment, B-problem, I-problem, B-test, I-test, <START>, <STOP>


In [0]:
#wordembeddings are the classic embeddings
# english_embedding = WordEmbeddings('en-glove')
english_embedding = WordEmbeddings('en')



2020-04-29 11:40:26,708 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmp377xgbr8


100%|██████████| 160000128/160000128 [00:03<00:00, 50886295.46B/s]

2020-04-29 11:40:29,983 copying /tmp/tmp377xgbr8 to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2020-04-29 11:40:30,369 removing temp file /tmp/tmp377xgbr8
2020-04-29 11:40:30,462 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim not found in cache, downloading to /tmp/tmpkmh359oj


100%|██████████| 21494764/21494764 [00:00<00:00, 46455063.82B/s]

2020-04-29 11:40:31,054 copying /tmp/tmpkmh359oj to cache at /root/.flair/embeddings/glove.gensim
2020-04-29 11:40:31,084 removing temp file /tmp/tmpkmh359oj



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
sequencetagger = SequenceTagger(hidden_size=128,
                                        embeddings=english_embedding,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

In [0]:
trainer = ModelTrainer(sequencetagger, corpus)

In [0]:
# patience is by default 3 
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150, monitor_test=True, embeddings_storage_mode ='gpu')

2020-04-29 11:41:12,795 ----------------------------------------------------------------------------------------------------
2020-04-29 11:41:12,796 Model: "SequenceTagger(
  (embeddings): WordEmbeddings('en-glove')
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=10, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2020-04-29 11:41:12,798 ----------------------------------------------------------------------------------------------------
2020-04-29 11:41:12,799 Corpus: "Corpus: 11422 train + 4893 dev + 27625 test sentences"
2020-04-29 11:41:12,800 ----------------------------------------------------------------------------------------------------
2020-04-29 11:41:12,802 Parameters:
2020-04-29 11:41:12,803  - learning_rate: "0.1"
2020-04-29 11:41:12,804  - m

/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


2020-04-29 11:41:18,031 epoch 1 - iter 35/357 - loss 7.75004229 - samples/sec: 216.38
2020-04-29 11:41:22,024 epoch 1 - iter 70/357 - loss 7.06569052 - samples/sec: 281.96
2020-04-29 11:41:26,578 epoch 1 - iter 105/357 - loss 6.50798885 - samples/sec: 247.09
2020-04-29 11:41:30,885 epoch 1 - iter 140/357 - loss 6.09738637 - samples/sec: 261.23
2020-04-29 11:41:35,467 epoch 1 - iter 175/357 - loss 5.81895655 - samples/sec: 245.71
2020-04-29 11:41:39,889 epoch 1 - iter 210/357 - loss 5.53458138 - samples/sec: 254.49
2020-04-29 11:41:44,525 epoch 1 - iter 245/357 - loss 5.31994035 - samples/sec: 242.59
2020-04-29 11:41:48,703 epoch 1 - iter 280/357 - loss 5.12053878 - samples/sec: 269.69
2020-04-29 11:41:53,434 epoch 1 - iter 315/357 - loss 4.98514322 - samples/sec: 238.22
2020-04-29 11:41:57,733 epoch 1 - iter 350/357 - loss 4.85167450 - samples/sec: 261.62
2020-04-29 11:41:58,690 ----------------------------------------------------------------------------------------------------
2020-04

{'dev_loss_history': [tensor(2.7825, device='cuda:0'),
  tensor(2.3154, device='cuda:0'),
  tensor(2.0501, device='cuda:0'),
  tensor(1.9109, device='cuda:0'),
  tensor(1.9561, device='cuda:0'),
  tensor(1.6659, device='cuda:0'),
  tensor(1.6093, device='cuda:0'),
  tensor(1.5244, device='cuda:0'),
  tensor(1.4786, device='cuda:0'),
  tensor(1.4824, device='cuda:0'),
  tensor(1.4258, device='cuda:0'),
  tensor(1.3320, device='cuda:0'),
  tensor(1.4113, device='cuda:0'),
  tensor(1.3018, device='cuda:0'),
  tensor(1.3333, device='cuda:0'),
  tensor(1.3124, device='cuda:0'),
  tensor(1.2760, device='cuda:0'),
  tensor(1.2263, device='cuda:0'),
  tensor(1.1834, device='cuda:0'),
  tensor(1.2294, device='cuda:0'),
  tensor(1.1906, device='cuda:0'),
  tensor(1.1716, device='cuda:0'),
  tensor(1.1540, device='cuda:0'),
  tensor(1.1256, device='cuda:0'),
  tensor(1.1346, device='cuda:0'),
  tensor(1.1013, device='cuda:0'),
  tensor(1.0833, device='cuda:0'),
  tensor(1.2150, device='cuda:0'),


In [0]:
result, _ = sequencetagger.evaluate([corpus.test])

RuntimeError: ignored

In [0]:
#this is CONll eval style - exact match 
print(result.detailed_results)

In [0]:
# TODO: implement inexact match evaluation - token level

In [0]:
model = SequenceTagger.load('resources/taggers/example-ner/final-model.pt')

2020-04-14 09:58:47,509 loading file resources/taggers/example-ner/final-model.pt


In [0]:
model.tag_type

'ner'

In [0]:
model.tag_type = 'predicted_ner'

# predict tags
pred = model.predict(corpus.test)

In [0]:
pred[36].to_tagged_string('predicted_ner')

'5.) LBBB'

In [0]:
pred[36].to_tagged_string('ner')

'5.) LBBB <B-problem>'